# Импорт библиотек

In [ ]:
import torch # <- база для нейронок
import torch.nn as nn
from tqdm import tqdm  # <- визуализируем полоску с процессом обучения (прогресс бар + факт: с такадум с арабского это прогресс :) ) 
from sklearn.model_selection import train_test_split # <- делит данные для тренировки и теста (в нашем случае валидации)
from torch.utils.data import Dataset, DataLoader # <- создает датасетики
from torchvision.transforms import v2
from PIL import Image # <- для работы с фото
import pandas as pd # <- пандас, база. Работаем с датасетом
import wandb
import os
from transformers import ViTImageProcessor, ViTForImageClassification # <- модель
from torcheval.metrics.functional import multiclass_f1_score # <- посчитать метрику

### Все обучение модели проводилось на Kaggle, на нем очень удобно работать с GPU (своего не имеем)

Но память на кагле не бесконечная, поэтому ее надо иногда очищать, освобожая место от ненужного

А без GPU нельзя?

- конечно можно, только вот VIT достаточно тяжелая модель и обучаться это все будет без GPU оооочень долго. Но зато какой результат дает VIT! 

In [ ]:
torch.cuda.empty_cache() # <- тут очищаем
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # <- а это надо для дальнейшего использования гпу

# Создание датасета

In [ ]:
class VITdataset(Dataset):
    def __init__(self, images_paths, images_names, images_indxes, trainable): 
        self.images_paths = images_paths # <- путь до папки с картинками (у каждого он свой)
        self.images_names = images_names # <- название картинок в папке
        self.images_indxes = images_indxes # <- метки классов, по задаче их 10
        self.processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224') # загружаем предобученную модель
        image_mean, image_std = self.processor.image_mean, self.processor.image_std
        normalize = v2.Normalize(mean=image_mean, std=image_std)

        # trainable - указывает тренировка или валидация
        if trainable == True:
            # Для обучения делаем аугментацию
            self.transform = v2.Compose([
                v2.Resize((self.processor.size["height"], self.processor.size["width"])),
                v2.RandomHorizontalFlip(0.4),
                v2.RandomVerticalFlip(0.1),
                v2.RandomApply(transforms=[v2.RandomRotation(degrees=(0, 90))], p=0.5),
                v2.RandomApply(transforms=[v2.ColorJitter(brightness=.3, hue=.1)], p=0.3),
                v2.RandomApply(transforms=[v2.GaussianBlur(kernel_size=(5, 9))], p=0.3),
                v2.ToTensor(),
                normalize
            ])
        elif trainable == False:
            # Здесь подгоняем картинку под требования модели (размер картинки и остальное)
            self.transform = v2.Compose([
                v2.Resize((self.processor.size["height"], self.processor.size["width"])),
                v2.ToTensor(),
                normalize
            ])

    # Чтобы понимать сколько элементов в эпохе, чтобы понять когда даталодеру закончить итерироваться
    def __len__(self):
        return len(self.images_indxes)

    # Тут возвращаем картинку и метку (класс), а даталодер сформирует батчи, в которых первый элемент - картинка, второй - массив из меток 
    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.images_paths, self.images_names[idx][0]))
        image = self.transform(image)
        return image, self.images_indxes[idx]
    

### Основные моменты для кода выше:

Почему VIT?

- предобучен на разных ImageNet-21k (в сумме около 15млн картинок!!)

Почему не с нуля, а предобученную? 

- сделает качество нашей будущей модели выше, за счет огромного количества данных, на которых была обучена модель


Зачем используем аугментацию?

- увеличим количество картинок для тренировки

- а еще! Аугментация сделает нашу модель более устойчивой к шумам и небольшим изменениям данных, это еще предотвратит переобучение


# Меняем голову

А зачем?

Во-первых, наша задача отличается от той, которую решала VIT, как минимум количеством классов, значит, надо что-то менять

Во-вторых, дообучим модель на наших данных (конкретно fine-tuning (существуют еще transfer learning и linear probing))

In [ ]:
class Head(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, input_dim) # <- линейный слой раз
        self.gelu1 = nn.GELU() # <- функция активации
        self.linear2 = nn.Linear(input_dim, output_dim) # <- линейный слой двас

    # Проход вперед
    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu1(x)
        x = self.linear2(x)

        return x

# Weights & Biases или же WandB

Для дальнейшего беспроблемного запуска кода нужно зарегистрироваться на платформе <Weights & Biases>, там можно смотреть на то, как падает лосс при увеличении эпох, и посмотреть на f1-метрику. Зарегистрировать нужно, чтобы получить API key

Прикладываю визуализацию последней версии модели (если смотреть через GitHub - фото будет недоступно, оно находится в <последняя модель.png>) 

<image src="последняя модель.png" alt="Текст с описанием картинки">


In [ ]:
wandb.login()
wandb.init(project='vit-image-classification')

# VIT

In [ ]:
class VIT():
    def __init__(self,):
        self.model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
        self.model.classifier = Head(self.model.classifier.in_features, 10) # <- меняем голову на нашу
        self.model = self.model.to(device) # <- перекладываем на GPUшку

    # Для обучения
    def train(self, dataset, validation, epochs):
        # Adam - метод оптимизации нейронки (для RNN и Transformers подходит), model.parameters() - передаем параметры, lr - длина шага град спуска, 
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-7)
        criterion = nn.CrossEntropyLoss() # <- функция потерь
        lambda_lr = lambda epoch: 0.4 ** epoch
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_lr)
        for epoch in range(epochs):
            self.model.train() # сейчас будет тренировать модель
            # передаем тензора на куду
            for images, targets in tqdm(dataset, desc='Training', colour="cyan"):
                images = images.to(device)
                targets = targets.to(device)

                optimizer.zero_grad() # зануляем старые градиенты, чтобы новые не суммировать со старыми
                model_output = self.model(images).logits
                loss = criterion(model_output, targets)
                loss.backward() # мы уже на проходе назад (а когда-то выше был проход вперед)
                optimizer.step() # делаем шаг градиентного спуска, то есть обновляем веса

                wandb.log({"loss": loss})
            scheduler.step()

            # Теперь оформим валидацию
            self.model.eval()
            F1_sum = []
            # на тестовой выборке нам не нужно градиенты, поэтому сделаем так, чтобы быстрее считалось
            with torch.no_grad():
                for images, targets in tqdm(validation, desc='Validation', colour="green"):
                    images = images.to(device)
                    targets = targets.to(device)

                    optimizer.zero_grad()
                    model_output = self.model(images).logits
                    pred_class = torch.argmax(model_output, dim=1)

                    F1_metric = multiclass_f1_score(pred_class, targets, num_classes=10)
                    F1_sum.append(F1_metric.item())

            F1_sum = sum(F1_sum)/len(F1_sum)
            wandb.log({"F1_metric": F1_sum})

            wandb.log({"epoch": epoch + 1})

        PATH = '/kaggle/working/Vit.pt'
        torch.save(self.model.state_dict(), PATH)

    # И вот мы уже на методе предсказания классов для теста
    def predict(self, path_to_images):
        names = os.listdir(path_to_images)
        processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
        image_mean, image_std = processor.image_mean, processor.image_std

        normalize = v2.Normalize(mean=image_mean, std=image_std)
        transform = v2.Compose([
            v2.Resize((processor.size["height"], processor.size["width"])),
            v2.ToTensor(),
            normalize
        ])

        model_result = []
        for name in tqdm(names, desc='Prediction', colour="red"):
            image = Image.open(os.path.join(path_to_images, name))
            image = transform(image)
            image = torch.unsqueeze(image, 0)
            image = image.to(device)
            pred_class = torch.argmax(self.model(image).logits).item()
            model_result.append(pred_class)

        # А вот созданием датасета с предсказаниями 
        output = pd.DataFrame({
            'image_name': names,
            'predicted_class': model_result
        })
        # Сохраняем!
        output.to_csv('/kaggle/working/submission11.csv', index=False)

# Теперь пришло время подготовить данные

In [ ]:
path = '/kaggle/input/train-zip/train'

data_idx = pd.read_csv('/kaggle/input/train-data/train.csv')
img_to_class = {row['image_name']: row['class_id'] for _, row in data_idx.iterrows()}
class_to_name = {row['class_id']: row['unified_class'] for _, row in data_idx.iterrows()}
images_names = data_idx['image_name'].values
imagest_target = [img_to_class[name] for name in images_names]
X_train, X_test, y_train, y_test = train_test_split(images_names.reshape(-1,1), imagest_target, test_size=0.2, stratify=imagest_target, random_state=42)

train = VITdataset(path, X_train, y_train, trainable = True)
val = VITdataset(path, X_test, y_test, trainable = False)

train = DataLoader(train, batch_size=16, shuffle=True)
val = DataLoader(val, batch_size=16, shuffle=False)

# Тренируем!

In [ ]:
NUM_EPOCHS = 7

Vit = VIT
Vit.train(train, val, NUM_EPOCHS)

# Предсказываем!

In [ ]:
pred_path = '/kaggle/input/test-zip/test'
Vit.predict(pred_path)

# Теперь можно посмотреть на датасет с предсказаниями

In [ ]:
df = pd.read_csv('/kaggle/working/submission11.csv')
df

Что помогло для решения задачи (и в принципе для погружения в ML/DL)

- курсы Евгения Соколова (МО1 и МО2)

- курс глубинного обучения от Ильдуса Садртдинова

лекции выше читались для студентов 3 курса ПМИ ВШЭ

- про VIT узнала, когда искала модели для улучшения решения на AIChallenge, 
но тогда остановилась на ResNet, поэтому в этот раз захотелось применить модель VIT и лучше в ней ней разобраться 